In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

In [6]:
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\dogs-vs-cats\\')
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'train\\train'
validation_data_dir = 'validation\\validation'
nb_train_samples = 102
nb_validation_samples = 40
epochs = 2
batch_size = 16

In [65]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator)
    
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        generator)
    
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [66]:
save_bottlebeck_features()

Found 102 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [77]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy',mode='rb'))
    train_labels = np.array(
        [0] * int((nb_train_samples / 2)) + [1] * int((nb_train_samples / 2)))

    validation_data = np.load(open('bottleneck_features_validation.npy',mode='rb'))
    validation_labels = np.array(
        [0] * int((nb_validation_samples / 2)) + [1] * int((nb_validation_samples / 2)))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    
    score = model.evaluate(train_data, train_labels, verbose=0) 
    print('Train score:', score[0]) 
    print('Train accuracy:', score[1])
    score = model.evaluate(validation_data, validation_labels, verbose=0) 
    print('Test score:', score[0]) 
    print('Test accuracy:', score[1])
    
    model.save_weights(top_model_weights_path)

In [79]:
train_top_model()

Train on 102 samples, validate on 40 samples
Epoch 1/2
102/102 [==============================] - 1s 7ms/step - loss: 2.9933 - acc: 0.5098 - val_loss: 3.6539 - val_acc: 0.5000
Epoch 2/2
102/102 [==============================] - 0s 2ms/step - loss: 1.3530 - acc: 0.6667 - val_loss: 1.6627 - val_acc: 0.5750
Train score: 0.6358694362581945
Train accuracy: 0.7352941182314181
Test score: 1.6627135753631592
Test accuracy: 0.575


In [ ]:
model.